In [1]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import poisson

import matplotlib.pyplot as plt

In [2]:
data_folder='~/Data/Intermediate_Output'

In [3]:
C=70
downweight=0.8

In [4]:
downweight_granularity=['dtime','Direction','dday','month']

# Data-based downweight factor

In [5]:
C_datadriven=pd.read_csv('~/Data/Intermediate_Output/R_Training_Pax.csv')
C_datadriven=C_datadriven.groupby(['DepDate']+downweight_granularity)['NumPax'].sum().reset_index()
C_datadriven=C_datadriven.groupby(downweight_granularity)['NumPax'].max().reset_index().rename(columns={'NumPax': 'downweight'})
C_datadriven['downweight']=C_datadriven['downweight'].astype('float')/C
C_datadriven['downweight']=C_datadriven['downweight'].apply(lambda x: downweight if x>downweight else x)
# C_datadriven['downweight']=downweight

C_datadriven['C']=C
C_datadriven['C_downweighted']=C*C_datadriven['downweight']

C_datadriven=C_datadriven[[x for x in C_datadriven.columns if x!='downweight']]

# Predicted data

In [6]:
Prediction_pax=pd.read_csv('~/Data/Intermediate_Output/R_Output_Training_Pax.csv',sep=',')

In [7]:
Prediction_pax=Prediction_pax[['Dprio']+downweight_granularity+['forecast_bookings']]\
.drop_duplicates(subset=['Dprio']+downweight_granularity)
Prediction_pax.index=range(Prediction_pax.shape[0])

Prediction_revenue=pd.read_csv('~/Data/Intermediate_Output/R_Output_Training_Revenue.csv',sep=',')
Prediction_revenue=Prediction_revenue.rename(columns={'forecast_bookings': 'forecast_revenue'})
Prediction_revenue=Prediction_revenue[['Dprio']+downweight_granularity+['forecast_revenue']]\
.drop_duplicates(subset=['Dprio']+downweight_granularity)
Prediction_revenue.index=range(Prediction_revenue.shape[0])

Prediction_pax=Prediction_pax.sort_values(by=downweight_granularity+['Dprio'])
Prediction_revenue=Prediction_revenue.sort_values(by=downweight_granularity+['Dprio'])

Prediction=pd.concat([Prediction_pax[[x for x in Prediction_pax.columns if x!='NumPax' and x!='forecast_node']],
                      Prediction_revenue['forecast_revenue']],axis=1)

Prediction['forecast_yield']=Prediction['forecast_revenue']/Prediction['forecast_bookings']

Prediction.index=range(Prediction.shape[0])

### Cumulations 

In [8]:
Prediction=Prediction.set_index(['Dprio']+downweight_granularity)

Intermediate_forecast_bookings=Prediction.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)])['forecast_bookings'].cumsum().reset_index()
Intermediate_forecast_bookings=Intermediate_forecast_bookings.rename(columns={'forecast_bookings': 'forecast_bookings_intermediate'})

Intermediate_forecast_revenue=Prediction.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)])['forecast_revenue'].cumsum().reset_index()
Intermediate_forecast_revenue=Intermediate_forecast_revenue.rename(columns={'forecast_revenue': 'forecast_revenue_intermediate'})

Prediction=Prediction.reset_index()
Prediction['Dprio']=-Prediction['Dprio']
Prediction=Prediction.set_index(['Dprio']+downweight_granularity)

Cumulation_forecast_bookings=Prediction.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)])['forecast_bookings'].cumsum().reset_index()
Cumulation_forecast_bookings['Dprio']=-Cumulation_forecast_bookings['Dprio']

Cumulation_forecast_revenue=Prediction.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)])['forecast_revenue'].cumsum().reset_index()
Cumulation_forecast_revenue['Dprio']=-Cumulation_forecast_revenue['Dprio']

Cumulation_forecast=Cumulation_forecast_bookings.merge(Cumulation_forecast_revenue,on=['Dprio']+downweight_granularity)
Cumulation_forecast=Cumulation_forecast.merge(Intermediate_forecast_bookings,on=['Dprio']+downweight_granularity)
Cumulation_forecast=Cumulation_forecast.merge(Intermediate_forecast_revenue,on=['Dprio']+downweight_granularity)

Prediction=Prediction.reset_index()
Prediction['Dprio']=-Prediction['Dprio']

In [9]:
Cumulation_forecast_365only=Cumulation_forecast.loc[Cumulation_forecast['Dprio']==365,:]
Cumulation_forecast_365only.index=range(Cumulation_forecast_365only.shape[0])

Cumulation_forecast_365only['TotalRevenue_over365_full']=C*(Cumulation_forecast_365only['forecast_revenue_intermediate']/Cumulation_forecast_365only['forecast_bookings_intermediate'])
Cumulation_forecast_365only['TotalRevenue_over365_downweighted']=C*downweight*(Cumulation_forecast_365only['forecast_revenue_intermediate']/Cumulation_forecast_365only['forecast_bookings_intermediate'])

Cumulation_forecast_365only=Cumulation_forecast_365only[downweight_granularity+['TotalRevenue_over365_full','TotalRevenue_over365_downweighted']]

/home/ubuntu/project-python/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/ubuntu/project-python/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


# Actual booking and revenue figures

In [10]:
ActualBookings=pd.read_csv('~/Data/Intermediate_Output/R_Test_Pax.csv')
ActualBookings=ActualBookings[['DepDate','Dprio']+downweight_granularity+['NumPax']]
ActualBookings['Dprio']=-ActualBookings['Dprio']
ActualBookings=ActualBookings.set_index(['Dprio']+downweight_granularity+['DepDate'])
ActualBookings=ActualBookings.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['NumPax'].cumsum().reset_index()
ActualBookings['Dprio']=-ActualBookings['Dprio']

In [11]:
ActualRevenue=pd.read_csv('~/Data/Intermediate_Output/R_Test_Revenue.csv')
ActualRevenue=ActualRevenue[['DepDate','Dprio']+downweight_granularity+['Coupons+YQ on current bookings']]
ActualRevenue['Dprio']=-ActualRevenue['Dprio']
ActualRevenue=ActualRevenue.set_index(['Dprio']+downweight_granularity+['DepDate'])
ActualRevenue=ActualRevenue.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)]+[len(downweight_granularity)+1])['Coupons+YQ on current bookings'].cumsum().reset_index()
ActualRevenue['Dprio']=-ActualRevenue['Dprio']

In [12]:
Data=ActualBookings.merge(ActualRevenue,on=['DepDate','Dprio']+downweight_granularity)
Data=Data.merge(C_datadriven,on=downweight_granularity)
Data=Data.merge(Cumulation_forecast,on=['Dprio']+downweight_granularity)
Data=Data.merge(Cumulation_forecast_365only,on=downweight_granularity)
IdealCurve=pd.read_csv(data_folder+'/IdealCurve.csv')
Data=Data.merge(IdealCurve,on=['Dprio']+downweight_granularity)

In [13]:
Data['NumPax_Remaining_full']=Data[['NumPax','Ideal_average_full']].apply(lambda x:max(x),axis=1)
Data['NumPax_Remaining_downweighted']=Data[['NumPax','Ideal_average_downweighted']].apply(lambda x:max(x),axis=1)
Data['expected_revenue_actual_full']=((Data['C']-Data['NumPax_Remaining_full'])*(Data['forecast_revenue_intermediate']/Data['forecast_bookings_intermediate']))\
+Data['Coupons+YQ on current bookings']
Data['expected_revenue_actual_downweighted']=((Data['C_downweighted']-Data['NumPax_Remaining_downweighted'])*(Data['forecast_revenue_intermediate']/Data['forecast_bookings_intermediate']))\
+Data['Coupons+YQ on current bookings']

In [14]:
Data=Data[['Dprio','DepDate']+downweight_granularity+['expected_revenue_actual_full',
    'expected_revenue_actual_downweighted','TotalRevenue_over365_full','TotalRevenue_over365_downweighted']]
Data=Data.rename(columns={'expected_revenue_actual_full': 'ActualRevenue_full', 
                          'expected_revenue_actual_downweighted': 'ActualRevenue_downweighted',
                          'TotalRevenue_over365_full': 'TotalRevenue_full',
                          'TotalRevenue_over365_downweighted': 'TotalRevenue_downweighted'})

In [15]:
deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Data['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Data['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Data=Data.merge(Map_DepTime,on='dtime')

Data['dtime']=Data['deptime']
Data=Data[[x for x in Data.columns if x!='deptime']]

In [16]:
Data['DepDate']=Data['DepDate'].apply(lambda x: pd.to_datetime(x))
Data['IssueDate']=Data['DepDate']-Data['Dprio'].apply(lambda x: datetime.timedelta(x))
Data=Data.loc[Data['IssueDate']<=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),:]
Data=Data[[x for x in Data.columns if x!='IssueDate']]

In [17]:
Data.to_csv('~/Data/FrontEnd_Input/GUI_in3.csv',index=False)